In [89]:
import random
import math
import copy

In [90]:
def read_tsp(filename):
    coords = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) == 3 and parts[0].isdigit():
                x, y = float(parts[1]), float(parts[2])
                coords.append((x, y))
    if not coords:
        raise ValueError("File .tsp không hợp lệ: không đọc được tọa độ")

    n = len(coords)
    dist_matrix = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                xi, yi = coords[i]
                xj, yj = coords[j]
                dist_matrix[i][j] = round(math.hypot(xi - xj, yi - yj))
    return dist_matrix

In [91]:
def read_knapsack(prefix):
    with open(f"/Users/Prj TTTH/EC-lab/dataset/knapsack/{prefix}_c.txt", "r") as f:
        capacity = int(f.readline().strip())
    with open(f"/Users/Prj TTTH/EC-lab/dataset/knapsack/{prefix}_p.txt", "r") as f:
        values = [int(x) for x in f.read().split()]
    with open(f"/Users/Prj TTTH/EC-lab/dataset/knapsack/{prefix}_w.txt", "r") as f:
        weights = [int(x) for x in f.read().split()]
    return weights, values, capacity

In [92]:
class Individual:
    def __init__(self, genome, skill_factor=None):
        self.genome = genome
        self.skill_factor = skill_factor
        self.fitness = None
    def copy(self):
        ind = Individual(self.genome.copy(), self.skill_factor)
        ind.fitness = self.fitness
        return ind


In [93]:
class TSP:
    def __init__(self, distance_matrix):
        self.distance_matrix = distance_matrix
        self.num_cities = len(distance_matrix)
    def decode(self, shared_genome):
        sorted_indices = sorted(range(len(shared_genome)),
                                key=lambda i: shared_genome[i])
        return sorted_indices[:self.num_cities]
    def evaluate(self, individual):
        genome = self.decode(individual.genome)
        dist = 0
        for i in range(self.num_cities):
            dist += self.distance_matrix[genome[i]][genome[(i+1) % self.num_cities]]
        individual.fitness = -dist
        return individual
    def generate_genome(self, L=51):
        return [random.random() for _ in range(L)]

In [94]:
# class Knapsack:
#     def __init__(self, weights, values, capacity):
#         self.weights = weights
#         self.values = values
#         self.capacity = capacity
#         self.num_items = len(weights)
#     def decode(self, shared_genome):
#         return [1 if g >= 0.5 else 0 for g in shared_genome[:self.num_items]]
#     def evaluate(self, individual):
#         genome = self.decode(individual.genome)
#         best_val = -1e9
#         n = len(genome)
#         for start in range(0, max(0, n - 15 + 1)):
#             sub = genome[start:start+15]
#             total_w = 0
#             total_v = 0
#             for i, g in enumerate(sub, start=start):
#                 if g == 1:
#                     total_w += self.weights[i]
#                     total_v += self.values[i]
#             if total_w <= self.capacity:
#                 best_val = max(best_val, total_v)
#         if best_val == -1e9:
#             best_val = -1e9
#         individual.fitness = best_val
#         return individual
#     def generate_genome(self, L=51):
#         return [random.random() for _ in range(L)]

In [95]:
import random

class Knapsack:
    def __init__(self, weights, values, capacity):
        self.weights = weights
        self.values = values
        self.capacity = capacity
        self.num_items = len(weights)

    def decode(self, genome):
        binary_genome = [0] * self.num_items
        remaining_capacity = self.capacity

        # Lấy (giá trị gene, index), sắp xếp theo gene giảm dần
        items_with_priority = [(genome[i], i) for i in range(min(len(genome), self.num_items))]
        items_with_priority.sort(key=lambda x: x[0], reverse=True)

        # Tham lam chọn item
        for _, idx in items_with_priority:
            if self.weights[idx] <= remaining_capacity:
                binary_genome[idx] = 1
                remaining_capacity -= self.weights[idx]

        return binary_genome

    def evaluate(self, individual):
        genome = self.decode(individual.genome)

        total_w = sum(w for g, w in zip(genome, self.weights) if g)
        total_v = sum(v for g, v in zip(genome, self.values) if g)

        if total_w > self.capacity:
            fitness = -1e9
        else:
            fitness = total_v

        individual.fitness = fitness
        return individual

    def generate_genome(self, L=51):
        return [random.random() for _ in range(L)]

In [96]:
# def tsp_crossover(p1, p2):
#     n = len(p1)
#     c1, c2 = p1.copy(), p2.copy()
#     i, j = sorted(random.sample(range(n), 2))
#     c1[i:j], c2[i:j] = p2[i:j], p1[i:j]
#     return c1, c2

In [97]:
def tsp_crossover(p1, p2):
    n = len(p1)

    perm1 = sorted(range(n), key=lambda i: p1[i])
    perm2 = sorted(range(n), key=lambda i: p2[i])

    i, j = sorted(random.sample(range(n), 2))

    def order_crossover(a, b):
        child = [-1] * n
        child[i:j] = a[i:j]
        pos = j
        for x in b:
            if x not in child:
                if pos >= n:
                    pos = 0
                child[pos] = x
                pos += 1
        return child

    c1_perm = order_crossover(perm1, perm2)
    c2_perm = order_crossover(perm2, perm1)

    def perm_to_genome(parent_genome, child_perm):
        values = [parent_genome[i] for i in child_perm]
        genome = [0.0] * n
        for idx, pos in enumerate(child_perm):
            genome[pos] = values[idx]
        return genome

    c1 = perm_to_genome(p1, c1_perm)
    c2 = perm_to_genome(p2, c2_perm)

    return c1, c2


In [98]:
def knapsack_crossover(p1, p2):
    point = random.randint(1, len(p1)-1)
    return p1[:point] + p2[point:], p2[:point] + p1[point:]

In [99]:
# def cross_task_crossover(p1, p2, alpha=0.5):
#     c1, c2 = [], []
#     for x, y in zip(p1, p2):
#         low = min(x, y) - alpha * abs(x-y)
#         high = max(x, y) + alpha * abs(x-y)
#         c1.append(random.uniform(low, high))
#         c2.append(random.uniform(low, high))
#     return c1, c2

In [100]:
def cross_task_crossover(p1, p2):
    c1, c2 = [], []
    for x, y in zip(p1, p2):
        avg = (x + y) / 2.0
        c1.append(avg)
        c2.append(avg)
    return c1, c2


In [101]:
def tsp_mutation(g):
    i, j = random.sample(range(len(g)), 2)
    g[i], g[j] = g[j], g[i]
    return g

In [102]:
def knapsack_mutation(g):
    i = random.randint(0, len(g)-1)
    g[i] = 1.0 - g[i]
    return g

In [103]:
class MFEA:
    def __init__(self, problems, pop_size=200, n_gen=1000,
                 crossover_rate=0.9, mutation_rate=0.1, rmp=0.3):
        self.problems = problems
        self.pop_size = pop_size
        self.n_gen = n_gen
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.rmp = rmp
        self.population = []

    def initialize(self):
        self.population = []
        # ratio = random.uniform(0.4, 0.6)
        ratio = 0.5
        n0 = int(self.pop_size * ratio)
        n1 = self.pop_size - n0
        for _ in range(n0):
            g = self.problems[0].generate_genome()
            self.population.append(Individual(g, 0))
        for _ in range(n1):
            g = self.problems[1].generate_genome()
            self.population.append(Individual(g, 1))
        random.shuffle(self.population)

    def evaluate(self):
        for ind in self.population:
            self.problems[ind.skill_factor].evaluate(ind)

    def run(self, return_pop=False):
        self.initialize()
        self.evaluate()

        best_by_skill = {0: None, 1: None}
        for ind in self.population:
            b = best_by_skill[ind.skill_factor]
            if b is None or (ind.skill_factor == 0 and ind.fitness < b.fitness) or (ind.skill_factor == 1 and ind.fitness > b.fitness):
                best_by_skill[ind.skill_factor] = ind.copy()

        best_history = []
        best_history.append((best_by_skill[0].fitness if best_by_skill[0] else None,
                             best_by_skill[1].fitness if best_by_skill[1] else None))

        for gen in range(self.n_gen):
            group0 = [ind for ind in self.population if ind.skill_factor == 0]
            group1 = [ind for ind in self.population if ind.skill_factor == 1]

            # sort & giữ lại 2 tốt nhất mỗi group
            group0.sort(key=lambda ind: ind.fitness, reverse=True)
            group1.sort(key=lambda ind: ind.fitness, reverse=True)
            elites = []
            elites.extend([ind.copy() for ind in group0[:2]])
            elites.extend([ind.copy() for ind in group1[:2]])

            new_pop = elites.copy()
            while len(new_pop) < self.pop_size:
                p1, p2 = random.sample(self.population, 2)

                # if p1.skill_factor == p2.skill_factor:
                #     if random.random() < self.crossover_rate:
                #         if p1.skill_factor == 0:
                #             c1g, c2g = tsp_crossover(p1.genome, p2.genome)
                #         else:
                #             c1g, c2g = knapsack_crossover(p1.genome, p2.genome)
                #     else:
                #         c1g, c2g = p1.genome.copy(), p2.genome.copy()

                #     if random.random() < self.mutation_rate:
                #         if p1.skill_factor == 0:
                #             c1g, c2g = tsp_mutation(c1g), tsp_mutation(c2g)
                #         else:
                #             c1g, c2g = knapsack_mutation(c1g), knapsack_mutation(c2g)

                #     children = [Individual(c1g, p1.skill_factor),
                #                 Individual(c2g, p1.skill_factor)]
                # else:
                #     if random.random() < self.rmp:
                #         c1g, c2g = cross_task_crossover(p1.genome, p2.genome)
                        
                #         children = [Individual(c1g, p1.skill_factor),
                #                     Individual(c2g, p2.skill_factor)]
                #     else:
                #         children = [p1.copy(), p2.copy()]

                if p1.skill_factor == p2.skill_factor:
                   
                    if random.random() < self.crossover_rate:
                        if p1.skill_factor == 0:
                            c1g, c2g = tsp_crossover(p1.genome, p2.genome)
                        else:
                            c1g, c2g = knapsack_crossover(p1.genome, p2.genome)
                    else:
                        c1g, c2g = copy.deepcopy(p1.genome), copy.deepcopy(p2.genome)

                    children = [
                        Individual(c1g, p1.skill_factor),
                        Individual(c2g, p1.skill_factor)
                    ]
                else:
                 
                    if random.random() < self.rmp:
                        c1g, c2g = cross_task_crossover(p1.genome, p2.genome)
                        children = [
                            Individual(copy.deepcopy(c1g), p1.skill_factor),
                            Individual(copy.deepcopy(c2g), p2.skill_factor)
                        ]
                    else:
                        children = [p1.copy(), p2.copy()]

                for child in children:
                    if random.random() < self.mutation_rate:
                        child.genome = copy.deepcopy(child.genome)
                        if child.skill_factor == 0:
                            child.genome = tsp_mutation(child.genome)
                        else:
                            child.genome = knapsack_mutation(child.genome)


                for c in children:
                    if len(new_pop) < self.pop_size:
                        new_pop.append(c)

            self.population = new_pop
            self.evaluate()

            # cập nhật best_by_skill
            for ind in self.population:
                b = best_by_skill[ind.skill_factor]
                if b is None:
                    best_by_skill[ind.skill_factor] = ind.copy()
                else:
                    if ind.fitness > b.fitness:
                        best_by_skill[ind.skill_factor] = ind.copy()

            best_history.append((best_by_skill[0].fitness if best_by_skill[0] else None,
                                 best_by_skill[1].fitness if best_by_skill[1] else None))

        if return_pop:
            return best_by_skill, best_history, self.population
        return best_by_skill, best_history

          

In [104]:
class Experiment:
    def __init__(self, algo, problems, seed_list=range(30), **algo_kwargs):
        self.algo = algo
        self.problems = problems if isinstance(problems, list) else [problems]
        self.seeds = seed_list
        self.algo_kwargs = algo_kwargs

    def run(self, verbose=True):
        all_best = []
        all_histories = []

        for i, seed in enumerate(self.seeds):
            random.seed(seed)
            algorithm = self.algo(self.problems, **self.algo_kwargs)
            best, history, population = algorithm.run(return_pop=True)

            best_per_problem = []
            for pid, prob in enumerate(self.problems):
                best_ind = None
                best_fit = -1e18
                for ind in population:
                    cand = ind.copy()
                    prob.evaluate(cand)
                    if cand.fitness > best_fit:
                        best_fit = cand.fitness
                        best_ind = cand
                best_per_problem.append(best_ind)

            all_best.append(best_per_problem)
            all_histories.append(history)

            if verbose:
                for pid, ind in enumerate(best_per_problem):
                    pname = type(self.problems[pid]).__name__
                    fitness_str = ind.fitness if ind is not None else None
                    print(f"[Seed {seed}] Best {pname} fitness = {fitness_str}")

        return all_best, all_histories

In [105]:
if __name__ == "__main__":
    tsp_file = "/Users/Prj TTTH/EC-lab/dataset/tsp/eil51.tsp"
    distance_matrix = read_tsp(tsp_file)
    tsp_problem = TSP(distance_matrix)

    weights, values, capacity = read_knapsack("p07")
    knapsack_problem = Knapsack(weights, values, capacity)

    print(tsp_problem.num_cities)
    print(knapsack_problem.num_items)

    problems = [tsp_problem, knapsack_problem]

    exp = Experiment(
        MFEA, problems,
        seed_list=range(30),
        pop_size=100, n_gen=500,
        crossover_rate=0.9, mutation_rate=0.1, rmp=0.3
    )

    results, all_histories = exp.run()

    best_tsp = None
  
    best_knap = None

    for best_per_problem in results:
        tsp_ind, knap_ind = best_per_problem

        if tsp_ind is not None and (best_tsp is None or tsp_ind.fitness > best_tsp.fitness):
            best_tsp = tsp_ind.copy()

        if knap_ind is not None and (best_knap is None or knap_ind.fitness > best_knap.fitness):
            best_knap = knap_ind.copy()

    print("\n===== FINAL RESULTS =====")
    if best_tsp:
        print(f"Best TSP fitness (tour length): {best_tsp.fitness}")
        print(f"Genome: {best_tsp.genome}")
    if best_knap:
        print(f"Best Knapsack fitness (value): {best_knap.fitness}")
        print(f"Genome: {best_knap.genome}")


51
15
[Seed 0] Best TSP fitness = -917
[Seed 0] Best Knapsack fitness = 1458
[Seed 1] Best TSP fitness = -905
[Seed 1] Best Knapsack fitness = 1455
[Seed 2] Best TSP fitness = -949
[Seed 2] Best Knapsack fitness = 1453
[Seed 3] Best TSP fitness = -986
[Seed 3] Best Knapsack fitness = 1455
[Seed 4] Best TSP fitness = -946
[Seed 4] Best Knapsack fitness = 1458
[Seed 5] Best TSP fitness = -968
[Seed 5] Best Knapsack fitness = 1458
[Seed 6] Best TSP fitness = -911
[Seed 6] Best Knapsack fitness = 1458
[Seed 7] Best TSP fitness = -997
[Seed 7] Best Knapsack fitness = 1458
[Seed 8] Best TSP fitness = -859
[Seed 8] Best Knapsack fitness = 1456
[Seed 9] Best TSP fitness = -878
[Seed 9] Best Knapsack fitness = 1450
[Seed 10] Best TSP fitness = -927
[Seed 10] Best Knapsack fitness = 1450
[Seed 11] Best TSP fitness = -960
[Seed 11] Best Knapsack fitness = 1454
[Seed 12] Best TSP fitness = -912
[Seed 12] Best Knapsack fitness = 1453
[Seed 13] Best TSP fitness = -843
[Seed 13] Best Knapsack fitness